<a href="https://colab.research.google.com/github/gnouvj/AQI_analysis/blob/main/Lat_Long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data.csv", index_col=0)
data.head()

,State,County,Year,Days_with_AQI,Good_Days,Moderate_Days,Unhealthy_for_Sensitive_Groups_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Max_AQI,90th_Percentile_AQI,Median_AQI,Days_CO,Days_NO2,Days_Ozone,Days_PM2_5,Days_PM10
0,Alabama,Baldwin,2013,273,235,38,0,0,0,0,87,54,36,0,0,200,73,0
1,Alabama,Clay,2013,118,100,18,0,0,0,0,65,52,32,0,0,0,118,0
2,Alabama,Colbert,2013,285,252,33,0,0,0,0,80,51,38,0,0,202,83,0
3,Alabama,DeKalb,2013,360,319,41,0,0,0,0,93,52,38,0,0,305,55,0
4,Alabama,Elmore,2013,244,229,15,0,0,0,0,87,48,36,0,0,244,0,0


In [ ]:
# 11176
longitude = []
lat = []
geolocator = Nominatim(user_agent="County_Conversion")
df = data.copy()

In [ ]:
df['County_State'] = df['County'] + ", " + df['State']

In [ ]:
County_State = df.County_State.unique()

In [ ]:
for i in County_State:
  location = geolocator.geocode(i, timeout = 10)
  if location is None:
    print(i)
    longitude.append(i)
    lat.append(i)

BAJA CALIFORNIA NORTE, Country Of Mexico
CHIHUAHUA STATE, Country Of Mexico
Mayagnez, Puerto Rico
Charles, Virginia
SONORA, Country Of Mexico


In [ ]:
print(lat.index('BAJA CALIFORNIA NORTE, Country Of Mexico')) #Mexico
print(lat.index('CHIHUAHUA STATE, Country Of Mexico')) #Mexico
print(lat.index('Mayagnez, Puerto Rico')) #Mayaguez
print(lat.index('Charles, Virginia')) #Charles City County
print(lat.index('SONORA, Country Of Mexico')) #Mexico

In [ ]:
df_new = data.copy()

In [ ]:
df_new['State'] = np.where(df_new['State'] == 'Country Of Mexico', 'Mexico', df_new['State'])
df_new['County'] = np.where(df_new['County'] == 'Mayagnez', 'Mayaguez', df_new['County'])
df_new['County'] = np.where((df_new["State"] == "Virginia") & (df_new["County"] == "Charles"), "Charles City County", df_new["County"])

In [ ]:
df_new['County_State'] = df_new['County'] + ", " + df_new['State']

In [ ]:
df_grouped = df_new.County_State

In [ ]:
LONG = []
LAT = []
count = 0
for i in range(len(df_grouped)):
  location = geolocator.geocode(f"{df_grouped[i]}", timeout = 10)
  if location is None:
    print(df_grouped[i])
  LONG.append(location.longitude)
  LAT.append(location.latitude)
  count += 1
  if count % 200 == 0:
    print(count)

200
400
600
800
1000


In [ ]:
LAT_PD = pd.Series(LAT)
LONG_PD = pd.Series(LONG)
df_G = pd.Series(df_grouped)

In [ ]:
df_grouped = pd.concat([df_G, LAT_PD, LONG_PD], axis = 1)

In [ ]:
df_grouped

,0,1,2
0,"Baldwin, Alabama",30.567753,-87.732439
1,"Clay, Alabama",33.242339,-85.819651
2,"Colbert, Alabama",34.647340,-87.792177
3,"DeKalb, Alabama",34.452435,-85.766212
4,"Elmore, Alabama",32.580123,-86.125195
...,...,...,...
1109,"Clay, South Dakota",42.919062,-96.971293
1110,"Beaverhead, Montana",45.170840,-112.953755
1111,"Glacier, Montana",48.696645,-112.946712
1112,"Hill, Montana",48.665578,-110.210710


In [ ]:
df_new['Latitude'] = df_new['County_State'].apply(lambda x: df_grouped[df_grouped[0] == x][1].values[0])
df_new['Longitude'] = df_new['County_State'].apply(lambda x: df_grouped[df_grouped[0] == x][2].values[0])

In [ ]:
df_new['LAT_LONG'] = df_new['Latitude'].astype(str) + ", " + df_new['Longitude'].astype(str)

In [ ]:
df_new.to_csv('map.csv', index=True)